# Model Structure

**Para-disk** is designed to produce model images of emission from molecular gas and/or dust in the disk around a young star. This calculation is performed assuming a parametric structure for the temperature, density, etc. Below we outline this parametric structure, and some of the assumptions that go into it. 

:::{important}

Within this code there are a couple of different options for the parametric structure. You may choose a different structure based on the type of system that you are modeling (e.g., protoplanetary disk vs. debris disk) or the complexity of the data (e.g., data with low spatial resolution data might warrant a simpler disk structure).

You choose different structure by specifying the version of disk.py that you use. The options are:
- **disk.py**: Protoplanetary disk structure, with a surface density that follows a power law with an exponential tail, and a vertical temperature gradient.
- **disk_pow.py**: Same as above, but the surface density only follows a power law, with no exponential tail at large radii.
- **disk_ecc.py**: Same as disk.py, but with an eccentric disk. This accounts for both changes in density and velocity with azimuth in the disk.
- **debris_disk.py**: A power law surface density, with a gas temperature that does not depend on height in the disk and is calculated using the equilibrium temperature of a blackbody grain. Instead of calculating the vertical density profile based on hydrostatic equilibrium, this code uses a specified value of the gas scale height. Intended for modeling dusty debris disks.
:::

## Disk Structure

### disk.py: Protoplanetary Disk

This model uses parametric forms of the temperature and density structure that are meant to represent protoplanetary disks, with a vertical temperature gradient, and a self-similar surface density distribution. 

The temperature follows a power law with radius, with a vertical temperature gradient connecting the cold midplane with the warm atmosphere:

$$T_{atm} = T_{atm0} \left(\frac{r}{150 au}\right)^{q}$$
$$T_{mid} = T_{mid0} \left(\frac{r}{150 au}\right)^{q}$$
$$T_{gas} = \begin{cases} T_{atm}+(T_{mid}-T_{atm})\left(cos\frac{\pi z}{2 Z_q}\right)^{2} \; \; \; \; &z<Z_q\\
                            T_{atm} \; \; \; \; &z>Z_q \end{cases}$$
$$Z_q = Z_{q0} \left(\frac{r}{150 au}\right)^{1.3}$$

The parameter $Z_q$ is the height above the midplane at which the gas temperature reaches its maximum value. Common values for $Z_{q0}$ might be a fixed number (e.g., 70 au, Rosenfeld et al. 2013) or a multiple of the pressure scale height (e.g., Dartois et al. 2003). 

The surface density follows a power law with an exponential tail, as expected for a viscously evolving disk (e.g., Lynden-Bell & Pringle 1974, Hartmann et al. 1998):

$$\Sigma_{gas}(r) = \frac{M_{gas}(2-\gamma)}{2 \pi R_c^2} \left(\frac{r}{R_c}\right)^{-\gamma}\exp\left[-\left(\frac{r}{R_c}\right)^{2-\gamma}\right]$$

where $M_{gas}$, $R_c$, and $\gamma$ are the gas mass, critical radius, and power law index. 

Once the surface density and temperature have been specified, the volume density is calculated using hydrostatic equilibrium:

$$-\frac{\partial\ln\rho}{\partial z} = \frac{\partial\ln T}{\partial z} = \frac{1}{c_s^2}\left[\frac{GM_*z}{(r^2+z^2)^{3/2}}\right]$$

$$c_s^2 = \frac{k_B T}{\mu m_h}$$

where $\mu$ is the mean molecular weight (=2.37). 


The velocity profile is Keplerian motion, with corrections for the height above the midplane and the pressure gradient.

$$\frac{v_{\phi}^2}{r} = \frac{GM_*r}{(r^2+z^2)^{3/2}}+\frac{1}{\rho_{gas}}\frac{\partial P_{gas}}{\partial r}$$

The line profile is assumed to be a Gaussian whose width ($\Delta V$) is set by the thermal and non-thermal motion. The non-thermal motion ($\delta v_{turb}$) can be taken as proportional to the local sound speed:

$$\Delta V = \sqrt{(2k_B T(r,z)/m_{CO})(1+\delta v_{turb}^2)}$$

or as a fixed velocity:

$$\Delta V = \sqrt{(2k_B T(r,z)/m_{CO})+\delta v_{turb}^2}$$


### disk_pow.py: Power-law protoplanetary disk

This model is identical to above, except that the surface density is assumed to follow a power law, without an exponential tail:

$$\Sigma_{gas} = \frac{M_{gas}(2-\gamma)}{2\pi (R_{out}^{2-\gamma}-R_{in}^{2-\gamma})}r^{-\gamma}$$



### disk_ecc.py: Eccentric protoplanetary disk

This model is similar to the protoplanetary disk model in disk.py, but it includes a prescription for eccentricity in the disk.

One modification comes in the surface density structure. The disk is treated as a series of concentric elliptical rings of semi-major axis $a$. The surface density is the same tapered power law as in disk.py, with the semi-major axis in place of the radial distance from the central star.

$$\Sigma_{gas}(a) = \frac{M_{gas}(2-\gamma)}{R_c^2} \left(\frac{a}{R_c}\right)^{-\gamma}\exp\left[-\left(\frac{a}{R_c}\right)^{2-\gamma}\right]$$

The surface density also needs a dependence on the angle from periapsis, $\phi$, since there will be a higher density near apastron, where the material spends most of its orbit, than at periastron, where the material spends less of its orbit. The density per unit length around a given orbit, $\lambda$, is given by

$$\lambda = \frac{m\sqrt{1-e^2}}{2\pi a(1+e\cos\phi)}$$

where $m$ is the mass contained within the ring. The linear density of the $i$th ring is related to the surface density through:

$$\Sigma(\phi) = \frac{\lambda_i(\phi)}{(r_{i+1}(\phi)-r_{i-1}(\phi))/2}$$

where $r$ is the distance from the central star (at one of the foci of the ellipse) as a function of semi-major axis and angle from periapsis.

$$r_i(\phi) = \frac{a_i(1-e^2)}{1+e\cos\phi}$$

The result is a surface density that depends on both semi-major axis and angle from periapsis:

$$\Sigma_{gas} = \frac{\Sigma_{gas}(a)(1-e^2)^{3/2}}{2\pi (1+e\cos\phi)^2}$$

In addition, the velocity profile is modified to account for the variations with $\phi$ (see derivation [here](https://pdfs.semanticscholar.org/75d1/c8533025d0a7c42d64a7fef87b0d96aba47e.pdf)).

$$v = \sqrt{\frac{GM_*}{a(1-e^2)}}\cos(\omega+\phi)+e\cos\omega$$

where $\omega$ in the angle of periastron (= the angle between periastron and the major axis). When multiplied by $\sin i$ this becomes the projected velocity along the line of sight. *Note that this does not include the correction for the height above the midplane, or for the pressure gradient.*


### debris_disk.py: Debris disk structure

This model is intended to predict the dust emission from a debris disk. It calculates the dust temperature assuming blackbody dust grains, based on the stellar luminosity and it assumes that the vertical density distribution has a Gaussian shape, with the scale height taken as a free parameter ($h = H/R$, or $h=H$, where $h$ is one of the model inputs). The surface density is a power law with radius (*note that the sign on $p$ is different than in the previous models*). This model only includes dust, and does not model the gas. 

$$T_d = \left( \frac{L_*}{16 \pi d^2 \sigma_B} \right)^{1/4}$$

$$\Sigma = \begin{cases} \Sigma_c r^p \; \; \; \; &r_i < r < r_o \\
                        0            \; \; \; \; &\rm{otherwise} \end{cases}$$
                        
$$\Sigma_c = \frac{M_{dust}(2+p)}{2\pi(R_{out}^{2+p}-R_{in}^{2+p})} $$

$$\rho_{dust} = \frac{\Sigma}{H\sqrt{\pi}}\exp\left(-\left(\frac{z}{H}\right)^2\right)$$

### Self-gravity

The codes disk.py and disk_pow.py can also include the self-gravity of the disk as a modification on the orbital velocity. This follows the prescription in Lodato 2007, Bertin & Lodato 1999 (used by Verosini et al. 2021 to model Elias 2-27). In the presence of self-gravity the orbital frequency becomes:
  
  $$\frac{v_{\phi}^2}{r} = \frac{GM_*r}{(r^2+z^2)^{3/2}}+\frac{1}{\rho_{gas}}\frac{\partial P_{gas}}{\partial r}+\frac{\partial \phi_{gas}}{\partial r}$$
  
  where $\phi_{gas}$ is the potential due to the self-gravity of the disk. This code does *not* include the modification to the vertical hydrostatic equilibrium due to self-gravity (e.g. eqn 18 of Rosenfeld et al. 2013). It only includes the modification to the velocity field. The gradient in the potential is calculated using:
  
  $$\frac{\partial \phi_{gas}}{\partial r}(r,z) = \frac{G}{r}\int^{\inf}_{0}\left[K(k)-\frac{1}{4}\left(\frac{k^2}{1-k^2}\right)\left(\frac{R}{r}-\frac{r}{R}+\frac{z}{rR}\right)E(k)\right]\sqrt{\frac{R}{r}}k\Sigma (R)dR$$
  
  where $k^2=4Rr/[(r+R)^2+z^2]$ and $E$ and $K$ are complete elliptic integrals of the first kind. When using self-gravity, you are encouraged to increase Rout to improve the accuracy of the integral above, especially if you are interested in the velocities near the outer edge of the disk. [*Only available in disk.py and disk_pow.py*]

## Radiative Transfer

To determine the emission, the radiative transfer equation must be integrated along each line of sight through the disk:

$$ I_{\nu} = \int^{\infty}_0 S_{\nu}(s)\exp[-\tau_{\nu}(s)]K_{\nu}(s)ds $$

where $\tau_{\nu}(s) = \int_0^s K_{\nu}(s')ds'$ is the optical depth, $K_{\nu}$ is the absorption coefficient, and $S_{\nu}$ is the source function. 

## CO abundance

This code was originally designed for modeling CO emission, and as such includes functionality for placing CO in the disk. Where CO is found in the disk is limited by freeze-out deep within the disk and photo-dissociation high in the disk. 

The upper edge of the CO distribution is calculated by vertically integrating the H nuclei density (0706$n_{\rm gas}$, following Rosenfeld et al. 2013, Qi et al. 2011, and Aikawa & Herbst 1999) and decreasing the CO abundance for heights, $z_{phot}$, where the column density is less than a specified threshold.

$$\sigma_s \gt 0.706\int_{z_{phot}}^{\infty}n_{gas}(r,z')dz'$$

This surface density can be writted in terms of the unitless $\Sigma_{21} = \sigma_s/1.59\times10^{21}$cm$^{-2}$. The code employs an upper and lower surface density boundary, specified in units of $\Sigma_{21}$. The upper boundary corresponds to photodissociation, while the lower boundary is often set to a large value (i.e., 1000). 

The CO abundance is assumed to be constant in regions of the disk where it is not photo-dissociated or frozen-out. Outside of these regions, the CO abundance is dropped by a large value (e.g., 5, 1e8).

### CO photo-desoprtion

Highly extended CO emission in systems like DM Tau and IM Lup can be difficult to explain given that CO should be frozen out in the outer diks. Instead, it is possible for photo-desorption, the process by which high-energy photons release CO that has been frozen onto dust grains back to the gas phase, to return a substantial amount of CO to the gas phase.

The code includes a prescription for adding in CO photo-desorption relies on increasing the CO abundance in a region between two column density boundaries, in a region of the disk where CO is frozen out. The argument for using column density boundaries is explained in detail in the appendix of Flaherty et al. 2020, and repeated here for convenience.

To accomodate CO photodesorption we compare the timescale for freeze-out, photodesorption, and photodissociation (the process by which high-energy photons break apart CO molecules into C and O), with CO photodesorption operating in the region where photodesorption operates more quickly than the other processes. 

The freeze-out timescale ($\tau_{fre}$) is set by the frequency at which CO molecules collide with dust particles (Flower et al. 2005). If we consider a gas particle moving at a thermal velocity $v_{th}$ through a medium containing dust particles of size $a_g$, the timescale for collisions is:

$$ \tau_{fre} = (n_g \pi a_g v_{th} S)^{-1} $$

where $n_g$ is the number density of grains, and $S$ is the efficiency, assumed to be 1. As the density decreases towards the outer disk, the freeze-out timescale increases, allowing CO to exist longer in the gas phase. 

CO photo-desorption is influenced by the UV radiation field, its ability to penetrate the disk, and the intrinsic efficiency with which CO desorbs from a dust grain in response to an incoming photon.

$$ R = I_{ISRF}\exp(-\gamma A_V)Y_{pd}a_g$$

where $R$ is the number of CO molecules that are photodesorbed per second per dust grain, $I_{ISRF}$ is the interstellar radiation field (=1 Habing = 10$^{8}$ photons s$^{-1}$ cm$^{-2}$; Habing 1968), $\gamma$ is a measure of the UV extinction relative to the visual extinction ($\sim$2 for small dust grains), $A_V$ is the visual extinction, and $Y_{pd}$ is the experimentally measured number of photodesorptions per UV photon. The term $I_{ISRF}\exp(-\gamma A_V)$ represents how many UV photons reach a certain depth within the disk; regions deep in the disk are more heavily shielded from UV radiation and are not subject to photodesorption. The factor $Y_{pd}$ can vary by $\sim$3-4 depending on the exact shape of the incident spectrum (Fayolle et al. 2011; Chen et al. 2014) and the temperature at which ice is deposited on the dust grains (Oberg et al. 2009), although we ignore these effects in our calculation and assume $Y_{pd}=2.7\times10^{-3}$. 

Given the photodesorption rate ($R$), the timescale the double the CO density is

$$ \tau_{pde} = \frac{n_{CO}}{Rn_g}$$

where $n_{CO}$ is the number density of CO molecules.

Once CO is released from the grains, it must survive as a molecule, without being destroyed by the same UV radiation that released it from the dust grain in the first place. The rate at which CO is photodissociated is (Visser et al. 2009):

$$k = \chi k_0 \Theta\exp(-\gamma A_V)$$

where $\chi$ is the radiation field in units of the Draine (1978) field (which corresponds to 1.7 Habings), $k_0$ is the unattenuated photodissociation rate (=$2.592\times10^{-10}$, in units of photodissociations per second per CO molecule), and $\Theta$ is the attenuation factor accounting for shielding of the radiation field by H$_2$ and self-shielding by CO. The photodissociation timescale is simply $\tau_{pdi} = 1/k$. 

Photodissociation and photodesorption have similar factors associated with the strength of the UV radiation field, with slightly different forms for representing the strength of the incident field, resulting in a similar increase in timescale towards larger depths in the disk. Photodissociation has an additional factor associated with self-shielding, making it more sensitive to the increase in density towards the midplane, leading photodissociation to become less effective more quickly than photodesorption. This allows for a narrow region in the outer disk above the midplane where photodesorption is more effective than photodissociation. 